In [1]:
import geojson
import json
import numpy as np 
from scipy.spatial import cKDTree
import pandas as pd  
from collections import deque 

In [ ]:
# convet geojson data into points and edges in csv format

with open('phaseOne.geojson', 'r') as file:
    geojson_data = geojson.load(file)
    lng = []
    lat = []
    ids = []
    featureLookup = {}
    adjacencyList = {}
    for i in range(len(geojson_data.features)):
        if geojson_data.features[i].geometry.type == "Point":
            # print(geojson_data.features[i])
            lng.append(geojson_data.features[i].geometry.coordinates[0])
            lat.append(geojson_data.features[i].geometry.coordinates[1])
            ids.append(geojson_data.features[i].id)

            # list for fast feature look up to retyrn a rout in format
            featureLookup[geojson_data.features[i].id] = geojson_data.features[i]

            # set up adjacencyList
            if geojson_data.features[i].id not in adjacencyList:
                adjacencyList[geojson_data.features[i].id]={
                    "coordinates":geojson_data.features[i].geometry.coordinates,
                    "edges":[]
                }
        
        if geojson_data.features[i].geometry.type == "LineString":
            # print(geojson_data.features[i])
            cur = geojson_data.features[i].properties

            # list for fast feature look up to retyrn a rout in format
            featureLookup[cur["key"]] = geojson_data.features[i]

            # set up edges
            adjacencyList[cur["from"]]["edges"].append(cur["to"])
            if cur["from"] not in adjacencyList[cur["to"]]["edges"]:
                adjacencyList[cur["to"]]["edges"].append(cur["from"])
    
    data = {
        'id':ids,
        'lat':lat,
        'lng':lng,
        }  
    df = pd.DataFrame(data)
    df.to_csv("points.csv", sep=',')
    

    with open('adjacencyList.json', 'w') as f:
        json.dump(adjacencyList, f)

    with open('featureLookup.json', 'w') as f:
        json.dump(featureLookup, f)


# with open('adjacencyList.json') as f:
#     data = json.load(f)
#     for i in data:
#         print(data[i])   
    
    

In [7]:
# Convert to radians
def latlon_to_cartesian(lat, lon):
    lat, lon = np.radians(lat), np.radians(lon)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.array([x, y, z]).T

def nearestPoint(lat,lng):
    data =  pd.read_csv('points.csv')
    target = np.array([lat,lng])
    tree = cKDTree(latlon_to_cartesian(data["lat"].to_numpy(), data["lng"].to_numpy()))
    _, idx = tree.query(latlon_to_cartesian(*target))
    # print(data.iloc[idx])
    return data.iloc[idx]['id']

In [ ]:
# import time

# start = time.time()
# nearestPoint(42.42158190520959,-76.49436752018303)
# end = time.time()
# print(end - start)

In [ ]:
# bfs to find shortest path
def bfs(start,end):

    file = open('adjacencyList.json', 'r')
    adjacencyList = json.load(file)
    file.close() 

    # maintain a queue of paths
    queue = []
    # visited set to prevent cycles
    visited = set()
    # push the first path into the queue
    queue.append([start])

    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        # get the last node from the path
        node = path[-1]
        # path found
        if node == end:
            return path
        # enumerate all adjacent nodes, construct a 
        # new path and push it into the queue
        edges = []
        if node in adjacencyList:
            edges = adjacencyList.get(node)["edges"]

        for nextN  in edges:
            if nextN not in visited:
                new_path = list(path)
                new_path.append(nextN)
                queue.append(new_path)
                visited.add(nextN)


In [23]:
# use path to create assemble geojson features to render on map
def pathToMapFeatures(path):

    file = open('featureLookup.json', 'r')
    featureLookup = json.load(file)
    file.close() 

    res = {
    "type": "FeatureCollection",
    "features": []
    }


    for i in range(len(path)-1):

        # add nodes
        node = featureLookup.get(path[i])
        res["features"].append(node)

        # add edges
        edgeKey = path[i]+"__"+path[i+1]
        edgeKey_alt = path[i+1]+"__"+path[i]

        edge ={}
        if edgeKey in featureLookup:
            edge = featureLookup.get(edgeKey)
        else:
            edge = featureLookup.get(edgeKey_alt)

        res["features"].append(edge)
    
    # adding last node
    node = featureLookup.get(path[-1])
    res["features"].append(node)

    return res

        


In [24]:
start = nearestPoint(42.420349,-76.498292)
end = nearestPoint(42.421880,-76.492024)

path = bfs(start,end)
print(path)

['n-1760678085442', 'n-1760678084163', 'n-1760678075748', 'n-1760678076994', 'n-1760678053429', 'n-1760678057068', 'n-1760678055906', 'n-1760678045806', 'n-1760678040307', 'n-1760678041787', 'n-1760678108427', 'n-1760678112999', 'n-1760678117435', 'n-1760677856227', 'n-1760677870372', 'n-1760677867740', 'n-1760678255464', 'n-1760678264531', 'n-1760678276305', 'n-1760678277570', 'n-1760678311760', 'n-1760678326795', 'n-1760678349227', 'n-1760678354669', 'n-1760678371971', 'n-1760678376312']


In [25]:

pathGeojson = pathToMapFeatures(path)



with open('pathGeojson.json', 'w') as f:
        json.dump(pathGeojson, f)